In [2]:
#importing the required libraries
import numpy as np
import pandas as pd
#libraries for plotting 
import seaborn as sns
import matplotlib.pyplot as plt
#libraries for neural network models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
################################################################################
# loading data
################################################################################
##reading Miller indeces of voxel-based RVEs for CNN model
#number of datapoints
num_of_RVEs=2000
#number of voxel per each side
dim=20
#directory of input files
path=''
# X is the input of the CNN model
X=np.zeros((num_of_RVEs,dim,dim,dim,3))
for RVE_num in np.arange(num_of_RVEs):
    #reading the Miller_index 
    Miller_idx=np.loadtxt(path+'Miller_index_RVE'+str(RVE_num+1)+'.csv',delimiter=',')
    X[RVE_num,:,:,:,:]=Miller_idx.reshape((1,dim,dim,dim,3))
    
##reading output of simulations (y)
y=np.loadtxt(path+'stress_output.csv',delimiter=',')

In [8]:
################################################################################
# splitting data into train and test batches
################################################################################
#import train_test_split module
from sklearn.model_selection import train_test_split
#splitting, here 25% of data are used for testing
X_train, X_test, y_train, y_test = train_test_split(X[:,:,:,:,0:2],y,test_size=0.25,random_state=10)

In [ ]:
################################################################################
# building the VGNNet model
################################################################################
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization

model = Sequential()

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', input_shape=(dim, dim, dim, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))
model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))
model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))



model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))


model.add(Dense(1, activation='relu'))
model.summary()

In [10]:
################################################################################
# compiling the model
################################################################################
## parameters fo training
initial_learning_rate = 0.0002
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

model.compile(loss="mse",
   optimizer=keras.optimizers.Adam(learning_rate=lr_schedule))

In [1]:
################################################################################
# training the model
################################################################################
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          epochs=100)

In [ ]:
## calculating loss
losses = pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
## Evaluation of the model
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
predictions = model.predict(X_test)
print(mean_absolute_error(y_test,predictions))
print(np.sqrt(mean_squared_error(y_test,predictions)))
print(explained_variance_score(y_test,predictions))

In [ ]:
## plotting performance for evaluation dataset
plt.figure(figsize=(9,9))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.xlabel('Actual stress (MPa)', fontsize=20)
plt.ylabel('Predicted stress (MPa)', fontsize=20)
plt.scatter(y_test,predictions)
plt.plot(y_test,y_test,'r')
plt.savefig('CNN_uni.jpg')

In [ ]:
## plotting performance for training dataset
plt.figure(figsize=(12,6))
predictions_train = model.predict(X_train)
plt.scatter(y_train,predictions_train)
plt.plot(y_train,y_train,'r')